### Bayesian Neural Network

In [ ]:
%matplotlib inline

import theano
import pymc3 as pm
import theano.tensor as T
import numpy as np

import matplotlib.pyplot as plt
import matplotlib.cm as cm

import seaborn as sns
from warnings import filterwarnings
filterwarnings('ignore') 
sns.set_style('white')

# image processing / scalers
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import scale

# dimensionality reduction
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

# neural network / machine learning
from sklearn.neural_network import MLPClassifier

# methods metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

# MNIST dataset
import keras.datasets as ds
import theano.tensor.nnet as nnet

### MNIST Dataset  / data processing

In [ ]:
# use this function for various processing steps
def process_dataset(dataset):
    processed_ds = []
    for image in dataset:
        processed_ds.append(image.flatten())
    return np.array(processed_ds)

In [ ]:
def scaler(dataset):
    #scaler = MinMaxScaler()
    return scale(dataset)

In [ ]:
fashion_mnist = ds.fashion_mnist

In [ ]:
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

In [ ]:
train_images = process_dataset(train_images)
test_images = process_dataset(test_images)

In [ ]:
print("shape of processed images: {}".format(train_images.shape))
print("shape of training labels: {}".format(train_labels.shape))

#### Binary Classification subset

In [ ]:
m1 = [(x, y) for x, y in zip(train_images, train_labels) if y == 3 or y == 0]
m2 = [(x, y) for x, y in zip(test_images, test_labels) if y == 3 or y == 0]

In [ ]:
binary_train_x, binary_train_y = map(list, zip(*m1))
binary_test_x, binary_test_y = map(list, zip(*m2))

binary_train_x = np.array(binary_train_x)
binary_train_y = np.array(binary_train_y)

binary_test_x = np.array(binary_test_x)
#binary_test_y = np.array(binary_test_y)

In [ ]:
print("binary train data size: {}".format(len(binary_train_x)))
print("binary test data size: {}".format(len(binary_test_x)))

print("binary train_label data size: {}".format(len(binary_train_y)))
print("binary test_label data size: {}".format(len(binary_test_y)))

#### Dimensionality Reduction

In [ ]:
def subset(training_images, training_labels, n):
    return training_images[0:n], training_labels[0:n]

In [ ]:
def reduce_dimensionality(data, labels, nfeatures, reducer):
        algorithm = ""
        if reducer == 'PCA':
            algorithm = PCA(n_components=nfeatures)
            algorithm.fit(data)
            data = algorithm.transform(data)
            print("explained_variance_ratio_ {}".format(algorithm.explained_variance_ratio_.sum()))
        
        if reducer == 'TSNE':
            data, labels = subset(data, train_labels, 500)
            algorithm = TSNE(n_components=2)
            data = algorithm.fit_transform(data)
            
        print("new total number of features {}".format(nfeatures))
        return data, labels

In [ ]:
binary_train_x, binary_train_y = reduce_dimensionality(binary_train_x, binary_train_y, 2, 'PCA')
binary_test_x, binary_test_y = reduce_dimensionality(binary_test_x, binary_test_y, 2, 'PCA')

print("shape of binary_train_x {}".format(binary_train_x.shape))
print("shape of binary_train_y {}".format(binary_train_y.shape))
print("shape of binary_test_x {}".format(binary_test_x.shape))
#print("shape of binary_test_y {}".format(binary_test_y.shape))

In [ ]:
binary_train_x = scaler(binary_train_x)

In [ ]:
binary_train_x[0]

In [ ]:
binary_test_x = scaler(binary_test_x)

In [ ]:
binary_test_x[0]

### Neural Network

In [ ]:
nn = MLPClassifier(hidden_layer_sizes=(5), max_iter=200, 
                    learning_rate='adaptive', solver='sgd', 
                    alpha=0.0002,  
                    activation='relu', verbose = True)

In [ ]:
nn.fit(binary_train_x, binary_train_y)

In [ ]:
predictions = nn.predict(binary_test_x)

In [ ]:
print("Result of simple neural network: {}\n".format(accuracy_score(binary_test_y, predictions)))
print(confusion_matrix(binary_test_y, predictions))

### Bayesian Neural Network

In [ ]:
binary_train_x, binary_train_y = subset(binary_train_x, binary_train_y, 500)
binary_test_x, binary_test_y = subset(binary_test_x, binary_test_y, 250)

print("binary train data size: {}".format(len(binary_train_x)))
print("binary test data size: {}".format(len(binary_test_x)))

print("binary train_label data size: {}".format(len(binary_train_y)))
print("binary test_label data size: {}".format(len(binary_test_y)))

In [ ]:
def construct_nn(ann_input, ann_output):
    n_hidden = 64
    
    # Initialize random weights between each layer
    init_1 = np.random.randn(binary_train_x.shape[1], n_hidden).astype(theano.config.floatX)
    init_2 = np.random.randn(n_hidden, n_hidden).astype(theano.config.floatX)
    init_out = np.random.randn(n_hidden).astype(theano.config.floatX)
        
    with pm.Model() as neural_network:
        # Weights from input to hidden layer
        weights_in_1 = pm.Normal('w_in_1', 0, sd=1, 
                                 shape=(binary_train_x.shape[1], n_hidden), 
                                 testval=init_1)
        
        # Weights from 1st to 2nd layer
        weights_1_2 = pm.Normal('w_1_2', 0, sd=1, 
                                shape=(n_hidden, n_hidden), 
                                testval=init_2)
        
        # Weights from hidden layer to output
        weights_2_out = pm.Normal('w_2_out', 0, sd=1, 
                                  shape=(n_hidden,), 
                                  testval=init_out)
        
        # Build neural-network using tanh activation function
        act_1 = nnet.relu(pm.math.dot(ann_input, 
                                         weights_in_1))
        act_2 = nnet.relu(pm.math.dot(act_1, 
                                         weights_1_2))
        act_out = pm.math.sigmoid(pm.math.dot(act_2, 
                                              weights_2_out))
        
        # Binary classification -> Bernoulli likelihood
        out = pm.Bernoulli('out', 
                           act_out,
                           observed=ann_output,
                           total_size=binary_train_y.shape[0] # IMPORTANT for minibatches
                          )
    return neural_network

ann_input = theano.shared(binary_train_x)
ann_output = theano.shared(binary_train_y)
neural_network = construct_nn(ann_input, ann_output)

In [ ]:
# from pymc3.theanof import set_tt_rng, MRG_RandomStreams
# set_tt_rng(MRG_RandomStreams(42))

In [ ]:
%%time

with neural_network:
    inference = pm.ADVI()
    approx = pm.fit(n=250000, method=inference)
    
trace = approx.sample(draws=5000)

In [ ]:
plt.plot(-inference.hist)
plt.ylabel('ELBO')
plt.xlabel('iteration');

#### Bayesian Neural Network Model Testing

In [ ]:
bnn_test_input = theano.shared(binary_test_x)
bnn_test_output = theano.shared(np.array(binary_test_y))

In [ ]:
neural_network = construct_nn(bnn_test_input, bnn_test_output)

In [ ]:
ann_input.set_value(binary_test_x)
ann_output.set_value(binary_test_y)

with neural_network:
    ppc = pm.sample_ppc(trace, samples=500, progressbar=False)

# Use probability of > 0.5 to assume prediction of class 1
pred = ppc['out'].mean(axis=0) > 0.5

In [ ]:
print('Accuracy = {}%'.format((binary_test_y == pred).mean() * 100))